# PizzeriaSimulation


En esta simulacion buscaremos simular una pizzeria, y seguiremos la siguente logica:un cliente ordena la pizza, el empleado recibe el pedido y empiza a hacer la o las pizzas,una vez hecho el pedido se tiene que hornar, el espacio del horno es limitado,el horno tambien tiene un tiempo que se tarda en hacer las pizzaspor lo que en ciertos momentos, los pedidos se quedaran esperando que el horno se desocupe espacios.para esta simulacion tendremos los recursos de epleado, y horno,los espacios del horno lo manejaremos con contenedores y la cantidad de masas tambien seria un contenedorrecordando que las masas se hacen el dia anterior por lo que se cuenta con una cantidad finita y se repone dia a dia como lo que estamos simulando es solo un dia de trabajo no recargaremos las masas, asumendo que el dia siguente yaseria es otra simulacion

### lógica:

1. **Orden del cliente**: Un cliente realiza un pedido de pizza.
2. **Preparación del pedido**: El empleado recibe el pedido y comienza a preparar la(s) pizza(s).
3. **Horneado**: Una vez preparadas, las pizzas deben hornearse. Sin embargo, el espacio en el horno es limitado, y cada horneado toma un tiempo determinado, En ciertos momentos, algunos pedidos tendrán que esperar hasta que se liberen espacios en el horno.

### Recursos

Para esta simulación, consideraremos los siguientes recursos:

- **Empleado**: Encargado de recibir y preparar los pedidos.
- **Horno**: Espacio limitado que se gestiona con contenedores para simular la capacidad disponible.

### Contenedores
- **Masa**: La cantidad de masa
- **Espacios en el horno**: La cantidad de espacios que el horno tiene disponibles


In [300]:
import simpy
import pandas as pd

import random

### Clase horno
En esta clase configuraremos las caracterista del horno, como seria el nombre, la capacidad del horno, la velocidad y los eventos del horno

In [301]:
class Oven:

    def __init__( self , env, name , baking_speed=8 , capacity_=5 ):

        self.env = env
        self.name = name
        self.baking_speed = baking_speed
        self.capacity= simpy.Container( env , init=capacity_ , capacity=capacity_)
        self.resource_Oven= simpy.Resource( env , capacity=capacity_ )

        self.dfO = pd.DataFrame({
            "name" : [name]
            , "baking_speed": [baking_speed]
            , "resource_capacity": [ capacity_ ]
        })

        self.dfE = pd.DataFrame()
        
    def bake_pizzas( self , number_pizzas, customer,employed ):
        while True:
            if self.capacity.level >= number_pizzas:
                yield self.capacity.get( number_pizzas )
                baking_time = self.baking_speed
                yield self.env.timeout(1)
                
                print( f" { self.env.now }: { self.name } is baking { number_pizzas } pizzas in { baking_time } minutes to {customer} " )
                df = pd.DataFrame({
                    "time": [ self.env.now ]
                    , "Oven": [ self.name ]
                    , "employed": [employed ]
                    , "customer" :[customer]
                    , "number_pizzas": [number_pizzas]
                    , "event": [ "baking pizzas" ]    
                })
    
                self.dfE = pd.concat( [self.dfE , df] )
                
                yield self.env.timeout( baking_time )
                yield self.capacity.put( number_pizzas )
                break
            
            print( f" { self.env.now }: { self.name } is ocuped {customer}!!!!!!!!!!!!!!!!!!!!!!!!!!!" )
            df = pd.DataFrame({
                    "time": [ self.env.now ]
                    , "Oven": [ self.name ]
                    , "employed": [employed ]
                    , "customer" :[customer]
                    , "number_pizzas": [number_pizzas]
                    , "event": [ "ocuped" ]    
                })
            self.dfE = pd.concat( [self.dfE , df] )

            yield self.env.timeout( 1 )
            
            

### Clase Empleado
En esta clase configuraremos las caracterista del Empleado, como seria el nombre, la velocidad y los eventos del Empleado

In [302]:
class Employed:
    def __init__(self,env,name,pizza_making_speed_pm):
        self.env = env
        self.name = name
        self.pizza_making_speed_pm = pizza_making_speed_pm
        self.resource_employed = simpy.Resource( env , capacity=1 )
        
        self.dfEp = pd.DataFrame({
            "name" : [self.name]
            , "pizza_making_speed_pm": [self.pizza_making_speed_pm]
        })
        
        self.dfE = pd.DataFrame()

    def make_pizza(self,doughs,oven,customer,number_pizzas):
        if doughs.level > number_pizzas:
            yield doughs.get( number_pizzas )
            pizza_making_time = self.pizza_making_speed_pm * number_pizzas
            yield self.env.timeout(1)
         
            print( f" { self.env.now }: { self.name } is meking { number_pizzas } pizzas in { pizza_making_time } minutes to {customer} " )
            df = pd.DataFrame({
                "time": [ self.env.now ]
                , "Oven": [ oven.name ]
                , "employed": [self.name]
                , "customer" :[customer]
                , "number_pizzas": [number_pizzas]
                , "event": [ "making pizzas" ]    
            })

            self.dfE = pd.concat( [self.dfE , df] )
            
            yield self.env.timeout( pizza_making_time )
            print( f" { self.env.now }: { self.name } is ready to bake { number_pizzas } pizzas in {oven.name},  to  {customer} " )

            with oven.resource_Oven.request() as request:
                yield request
                yield self.env.process( oven.bake_pizzas( number_pizzas, customer,self.name) )
                print( f" { self.env.now }:  { self.name } finished bake { number_pizzas } pizzas to { customer }  " )
                yield self.env.timeout( 1 )
        
        else :
            print( f" { self.env.now }: { self.name } does not have doughs!!!! for  {customer} " )
            df = pd.DataFrame({
                "time": [ self.env.now ]
                , "Oven": [ oven.name ]
                , "employed": [self.name]
                , "customer" :[customer]
                , "number_pizzas": [number_pizzas]
                , "event": [ "not have doughs" ]    
            })
            self.dfE = pd.concat( [self.dfE , df] )
      

### Clase Cliente
En esta clase configuraremos las caracterista del Cliente, como seria el nombre, la cantidad de pizzas a ordenar y los eventos del Empleado

In [303]:
class Customer:
    def __init__(self , env , name  , number_pizzas,oven,employed):
        self.env = env
        self.name = name
        self.number_pizzas= number_pizzas
        self.oven=oven
        self.employed=employed
        self.dfC = pd.DataFrame({
            "name" : [self.name]
            , "number_pizzas": [self.number_pizzas]
            , "employed_name": [ self.employed.name ]
        })

        self.dfE = pd.DataFrame()
        
    def order_pizza(self,doughs):
        
        print( f" { self.env.now }:  { self.name } is waiting to be attended " )
        df = pd.DataFrame({
            "time": [ self.env.now ]
            , "Oven": [ self.oven.name ]
            , "employed": [self.employed.name ]
            , "customer" :[self.name]
            , "number_pizzas": [self.number_pizzas]
            , "event": [ "waiting to be attended" ]
        })
        self.dfE = pd.concat( [self.dfE , df] )
        with self.employed.resource_employed.request() as request:
            yield request
            print( f" { self.env.now }:  { self.name } is ready to be attended by the { self.employed.name } " )
            yield self.env.process( self.employed.make_pizza( doughs,self.oven,self.name,self.number_pizzas) )
            print( f" { self.env.now }:  { self.name } was attended and leave the Pizzeria " )
            df = pd.DataFrame({
                "time": [ self.env.now ]
                , "Oven": [ self.oven.name ]
                , "employed": [self.employed.name ]
                , "customer" :[self.name]
                , "number_pizzas": [self.number_pizzas]
                , "event": [ "finished" ]    
            })
            self.dfE = pd.concat( [self.dfE , df] )
            yield self.env.timeout( 1 )
    

### Clase Pizzeria 
En esta clase es donde colocaremos la parametrizacion de la pizzera en general, la cantidad de empleados, hornos, clientes que tendra la simulacion

In [304]:
class Pizzeria:
    def __init__(self,env,num_employed,num_customers,num_oven,num_doughs):
        self.env=env
        self.num_customers=num_customers
        self.employees=[ Employed(
            env
            , f"Employed { i }"
            , random.randint( 4,8 )
        ) for i in range(num_employed) ]
        self.customers=[]
        self.ovens= [ Oven(
            env
            , f"Oven { i }"
            , random.randint( 8,10 )
            , random.randint( 5,10 )
        ) for i in range(num_oven) ]
        self.doughs=simpy.Container( env , init=num_doughs , capacity=num_doughs  )

        env.process( self.customers_arriving() )
        
    def customers_arriving(self):
        for i in range( self.num_customers):
            self.customers.append(Customer(
                env
                , f"Customer {i}"
                , random.randint( 1, 4 )
                , random.choice( self.ovens )
                , random.choice( self.employees )
            ))
            env.process( self.customers[i].order_pizza(self.doughs))
            yield self.env.timeout( random.randint( 1 , 20 ) ) ## Las personas llegan aleatoriamente entre 1 - 30 Minutos


## Iniciamos la simulacións

In [305]:
random.seed(111)

env = simpy.Environment()

pizzeria = Pizzeria( env , num_employed=2,num_customers=50,num_oven=1,num_doughs=150 )

env.run( until=480 ) ## 8 horas 

 0:  Customer 0 is waiting to be attended 
 0:  Customer 0 is ready to be attended by the Employed 0 
 1: Employed 0 is meking 4 pizzas in 20 minutes to Customer 0 
 7:  Customer 1 is waiting to be attended 
 21: Employed 0 is ready to bake 4 pizzas in Oven 0,  to  Customer 0 
 22:  Customer 2 is waiting to be attended 
 22: Oven 0 is baking 4 pizzas in 9 minutes to Customer 0 
 22:  Customer 2 is ready to be attended by the Employed 1 
 23: Employed 1 is meking 4 pizzas in 24 minutes to Customer 2 
 31:  Employed 0 finished bake 4 pizzas to Customer 0  
 32:  Customer 0 was attended and leave the Pizzeria 
 33:  Customer 1 is ready to be attended by the Employed 0 
 34: Employed 0 is meking 4 pizzas in 20 minutes to Customer 1 
 35:  Customer 3 is waiting to be attended 
 37:  Customer 4 is waiting to be attended 
 47: Employed 1 is ready to bake 4 pizzas in Oven 0,  to  Customer 2 
 48: Oven 0 is baking 4 pizzas in 9 minutes to Customer 2 
 51:  Customer 5 is waiting to be attended 


## recolectamos el dataframe de hornos y empezamos a unir el dataframe de Eventos

In [306]:
df_events = pd.DataFrame()

In [307]:
df_ovens = pd.DataFrame()

for p in pizzeria.ovens:
    df_ovens = pd.concat( [ df_ovens , p.dfO ] )
    df_events = pd.concat( [ df_events , p.dfE ] )

In [308]:
df_ovens

,name,baking_speed,resource_capacity
0,Oven 0,9,6


In [309]:
df_events

,time,Oven,employed,customer,number_pizzas,event
0,22,Oven 0,Employed 0,Customer 0,4,baking pizzas
0,48,Oven 0,Employed 1,Customer 2,4,baking pizzas
0,54,Oven 0,Employed 0,Customer 1,4,ocuped
0,55,Oven 0,Employed 0,Customer 1,4,ocuped
0,56,Oven 0,Employed 0,Customer 1,4,ocuped
0,58,Oven 0,Employed 0,Customer 1,4,baking pizzas
0,73,Oven 0,Employed 1,Customer 3,2,baking pizzas
0,76,Oven 0,Employed 0,Customer 5,1,baking pizzas
0,104,Oven 0,Employed 1,Customer 4,3,baking pizzas
0,104,Oven 0,Employed 0,Customer 7,3,baking pizzas


## recolectamos el dataframe de clientes y empezamos a unir el dataframe de Eventos

In [310]:
df_customers = pd.DataFrame()

for p in pizzeria.customers:
    df_customers = pd.concat( [ df_customers , p.dfC ] )
    df_events = pd.concat( [ df_events , p.dfE ] )

In [311]:
df_customers

,name,number_pizzas,employed_name
0,Customer 0,4,Employed 0
0,Customer 1,4,Employed 0
0,Customer 2,4,Employed 1
0,Customer 3,2,Employed 1
0,Customer 4,3,Employed 1
0,Customer 5,1,Employed 0
0,Customer 6,1,Employed 1
0,Customer 7,3,Employed 0
0,Customer 8,4,Employed 1
0,Customer 9,4,Employed 0


In [312]:
df_events

,time,Oven,employed,customer,number_pizzas,event
0,22,Oven 0,Employed 0,Customer 0,4,baking pizzas
0,48,Oven 0,Employed 1,Customer 2,4,baking pizzas
0,54,Oven 0,Employed 0,Customer 1,4,ocuped
0,55,Oven 0,Employed 0,Customer 1,4,ocuped
0,56,Oven 0,Employed 0,Customer 1,4,ocuped
...,...,...,...,...,...,...
0,430,Oven 0,Employed 0,Customer 39,4,waiting to be attended
0,439,Oven 0,Employed 1,Customer 40,2,waiting to be attended
0,452,Oven 0,Employed 1,Customer 41,1,waiting to be attended
0,457,Oven 0,Employed 1,Customer 42,4,waiting to be attended


## recolectamos el dataframe de empleados y empezamos a unir el dataframe de Eventos

In [313]:
df_employees = pd.DataFrame()

for p in pizzeria.employees:
    df_employees = pd.concat( [ df_employees , p.dfEp ] )
    df_events = pd.concat( [ df_events , p.dfE ] )

In [314]:
df_employees

,name,pizza_making_speed_pm
0,Employed 0,5
0,Employed 1,6


In [315]:
df_events

,time,Oven,employed,customer,number_pizzas,event
0,22,Oven 0,Employed 0,Customer 0,4,baking pizzas
0,48,Oven 0,Employed 1,Customer 2,4,baking pizzas
0,54,Oven 0,Employed 0,Customer 1,4,ocuped
0,55,Oven 0,Employed 0,Customer 1,4,ocuped
0,56,Oven 0,Employed 0,Customer 1,4,ocuped
...,...,...,...,...,...,...
0,328,Oven 0,Employed 1,Customer 24,3,making pizzas
0,360,Oven 0,Employed 1,Customer 33,4,making pizzas
0,406,Oven 0,Employed 1,Customer 34,4,making pizzas
0,443,Oven 0,Employed 1,Customer 35,1,making pizzas


## Analisis

empezaremos analizando cuanto tiempo estuvo el horno atrazando los pedidos por que estaba ocupado

In [316]:
sampleOcuped = df_events.loc[
    df_events["event"] == "ocuped"
    , :
].sort_values(  "time" )

In [317]:
sampleOcuped

,time,Oven,employed,customer,number_pizzas,event
0,54,Oven 0,Employed 0,Customer 1,4,ocuped
0,55,Oven 0,Employed 0,Customer 1,4,ocuped
0,56,Oven 0,Employed 0,Customer 1,4,ocuped
0,271,Oven 0,Employed 0,Customer 19,3,ocuped
0,272,Oven 0,Employed 0,Customer 19,3,ocuped
0,273,Oven 0,Employed 0,Customer 19,3,ocuped
0,274,Oven 0,Employed 0,Customer 19,3,ocuped
0,275,Oven 0,Employed 0,Customer 19,3,ocuped
0,276,Oven 0,Employed 0,Customer 19,3,ocuped
0,277,Oven 0,Employed 0,Customer 19,3,ocuped


In [318]:
sampleOcuped = df_events.loc[
    df_events["event"] == "ocuped"
    , :
].sort_values(  "time"   
).groupby(
    ["customer"]
    , as_index=False
).agg({
     "time": ["count"]
})

In [319]:
sampleOcuped

,customer,time
,,count
0,Customer 1,3
1,Customer 19,10
2,Customer 24,1
3,Customer 27,1
4,Customer 33,9


como vemos sololamente 5 clientes se atrazaron por el horno ocupado, y en general fue poco tiempo 
por lo que podemos deducir que en esta simulacion otro horno no haria mucha diferencia en la atencion de los pedidos

ahora analizaremos cuanto tiempo nos tardamos en atender a los clientes desde que 
ordenan la pizzas hasta que se finaliza la entrega de las pizzas

In [320]:
sample2 = df_events.sort_values(  ["customer","time"]   )

vamos a tener que hacer varias limpiezas a los datos por que queremos saver el tiempo promedio de atencion y no debemos tomar en cuenta los clientes que no tubieron los 3 eventos (	waiting to be attended, baking pizzas,finished), por que si no esta el evento baking pizzas y si estan los otros dos eventos, significa que no habian masas y realmente no se atendio la persona, y si no esta el evento finished pero si los otros dos,significa que no se le termino de atender en lo que se hizo la simulacion, para estos casos no se tomaran en cuenta los tiempos para el calculo del tiempo promedio de atencion

In [321]:
sample2.loc[:]

,time,Oven,employed,customer,number_pizzas,event
0,0,Oven 0,Employed 0,Customer 0,4,waiting to be attended
0,1,Oven 0,Employed 0,Customer 0,4,making pizzas
0,22,Oven 0,Employed 0,Customer 0,4,baking pizzas
0,32,Oven 0,Employed 0,Customer 0,4,finished
0,7,Oven 0,Employed 0,Customer 1,4,waiting to be attended
...,...,...,...,...,...,...
0,170,Oven 0,Employed 1,Customer 8,4,finished
0,85,Oven 0,Employed 0,Customer 9,4,waiting to be attended
0,116,Oven 0,Employed 0,Customer 9,4,making pizzas
0,137,Oven 0,Employed 0,Customer 9,4,baking pizzas


quitamos hacemos un flitro para que solo aparezcan los 3 eventos que ocupamos

In [322]:
df_filtered = sample2[sample2['event'].isin(['waiting to be attended', 'finished','baking pizzas'])]

In [323]:
df_filtered

,time,Oven,employed,customer,number_pizzas,event
0,0,Oven 0,Employed 0,Customer 0,4,waiting to be attended
0,22,Oven 0,Employed 0,Customer 0,4,baking pizzas
0,32,Oven 0,Employed 0,Customer 0,4,finished
0,7,Oven 0,Employed 0,Customer 1,4,waiting to be attended
0,58,Oven 0,Employed 0,Customer 1,4,baking pizzas
...,...,...,...,...,...,...
0,160,Oven 0,Employed 1,Customer 8,4,baking pizzas
0,170,Oven 0,Employed 1,Customer 8,4,finished
0,85,Oven 0,Employed 0,Customer 9,4,waiting to be attended
0,137,Oven 0,Employed 0,Customer 9,4,baking pizzas


sobre el filtro anterior ahora filtraremos de nuevo para que solo aparezcan los customers que tengan los 3 eventos

In [324]:
#Cuenta las ocurrencias de cada cliente
customer_counts = df_filtered['customer'].value_counts()

#Filtra los clientes que aparecen más de 2 veces
customers_more_than_3 = customer_counts[customer_counts >= 3].index

#Filtra el DataFrame para incluir solo esos clientes
df_filtered = df_filtered[df_filtered['customer'].isin(customers_more_than_3)]


ahora para el calculo del tiempo solo dejaremos los 2 eventos que ocupamos para el calculo del tiempo que seria waiting to be attended y finished

In [325]:
df_filtered =df_filtered[df_filtered['event'].isin(['waiting to be attended', 'finished'])]

In [326]:
df_filtered

,time,Oven,employed,customer,number_pizzas,event
0,0,Oven 0,Employed 0,Customer 0,4,waiting to be attended
0,32,Oven 0,Employed 0,Customer 0,4,finished
0,7,Oven 0,Employed 0,Customer 1,4,waiting to be attended
0,68,Oven 0,Employed 0,Customer 1,4,finished
0,86,Oven 0,Employed 1,Customer 10,2,waiting to be attended
...,...,...,...,...,...,...
0,114,Oven 0,Employed 0,Customer 7,3,finished
0,69,Oven 0,Employed 1,Customer 8,4,waiting to be attended
0,170,Oven 0,Employed 1,Customer 8,4,finished
0,85,Oven 0,Employed 0,Customer 9,4,waiting to be attended


In [327]:
df_filtered["lead"] = df_filtered.groupby("customer")["time"].shift(-1)

df_filtered[ "time_on_event"] = df_filtered["lead"] - df_filtered["time"]

df_filtered = df_filtered[df_filtered['event'].isin(['waiting to be attended'])]

In [328]:
df_filtered

,time,Oven,employed,customer,number_pizzas,event,lead,time_on_event
0,0,Oven 0,Employed 0,Customer 0,4,waiting to be attended,32.0,32.0
0,7,Oven 0,Employed 0,Customer 1,4,waiting to be attended,68.0,61.0
0,86,Oven 0,Employed 1,Customer 10,2,waiting to be attended,195.0,109.0
0,104,Oven 0,Employed 0,Customer 11,1,waiting to be attended,165.0,61.0
0,119,Oven 0,Employed 1,Customer 12,2,waiting to be attended,220.0,101.0
0,138,Oven 0,Employed 0,Customer 13,4,waiting to be attended,198.0,60.0
0,143,Oven 0,Employed 1,Customer 14,2,waiting to be attended,245.0,102.0
0,145,Oven 0,Employed 1,Customer 15,4,waiting to be attended,282.0,137.0
0,148,Oven 0,Employed 1,Customer 16,1,waiting to be attended,301.0,153.0
0,156,Oven 0,Employed 0,Customer 17,2,waiting to be attended,221.0,65.0


teniendo el tiempo ya solo nos faltaria calcular el promedio de de mitos en antencion 

In [329]:
average_time_on_event = df_filtered['time_on_event'].mean()

In [330]:
average_time_on_event

95.93939393939394

como observamos en promedio nos dio 95 minutos, siento este un tiempo muy alto, entonces lo que vamos a hacer es aumentar ya sea la cantidad de hornos o la cantidad de empleados para ver como bajamos este promedio a el estandar en pizzerias de 30 minutos

In [331]:
random.seed(107)

env = simpy.Environment()

pizzeria = Pizzeria( env , num_employed=4,num_customers=50,num_oven=2,num_doughs=150 )

env.run( until=480 ) ## 8 horas 

 0:  Customer 0 is waiting to be attended 
 0:  Customer 0 is ready to be attended by the Employed 2 
 1: Employed 2 is meking 1 pizzas in 8 minutes to Customer 0 
 9: Employed 2 is ready to bake 1 pizzas in Oven 1,  to  Customer 0 
 10: Oven 1 is baking 1 pizzas in 8 minutes to Customer 0 
 18:  Employed 2 finished bake 1 pizzas to Customer 0  
 19:  Customer 0 was attended and leave the Pizzeria 
 20:  Customer 1 is waiting to be attended 
 20:  Customer 1 is ready to be attended by the Employed 2 
 21: Employed 2 is meking 1 pizzas in 8 minutes to Customer 1 
 27:  Customer 2 is waiting to be attended 
 27:  Customer 2 is ready to be attended by the Employed 1 
 28: Employed 1 is meking 2 pizzas in 16 minutes to Customer 2 
 29: Employed 2 is ready to bake 1 pizzas in Oven 1,  to  Customer 1 
 30: Oven 1 is baking 1 pizzas in 8 minutes to Customer 1 
 38:  Employed 2 finished bake 1 pizzas to Customer 1  
 39:  Customer 1 was attended and leave the Pizzeria 
 44: Employed 1 is ready

In [332]:
df_ovens = pd.DataFrame()
df_customers = pd.DataFrame()
df_employees = pd.DataFrame()
df_events = pd.DataFrame()

for p in pizzeria.ovens:
    df_ovens = pd.concat( [ df_ovens , p.dfO ] )
    df_events = pd.concat( [ df_events , p.dfE ] )
    
for p in pizzeria.employees:
    df_employees = pd.concat( [ df_employees , p.dfEp ] )
    df_events = pd.concat( [ df_events , p.dfE ] )

for p in pizzeria.customers:
    df_customers = pd.concat( [ df_customers , p.dfC ] )
    df_events = pd.concat( [ df_events , p.dfE ] )

df_filtered = df_events[df_events['event'].isin(['waiting to be attended', 'finished','baking pizzas'])]

customer_counts = df_filtered['customer'].value_counts()

customers_more_than_3 = customer_counts[customer_counts >= 3].index

df_filtered = df_filtered[df_filtered['customer'].isin(customers_more_than_3)]

df_filtered["lead"] = df_filtered.groupby("customer")["time"].shift(-1)
df_filtered[ "time_on_event"] = df_filtered["lead"] - df_filtered["time"]
df_filtered = df_filtered[df_filtered['event'].isin(['waiting to be attended'])]
average_time_on_event = df_filtered['time_on_event'].mean()


In [333]:
average_time_on_event

38.56410256410256

aumentando 1 horno y 2 empleados mas el promedio de atencion mejora bastante, ya sercano al promedio en general de las pizzerias, siendo este la mejor configuracion para una pizzeria